# PRÁTICA GUIADA: Otimização de modelos II

## Introdução

#### Na aula passada abordamos um processo de busca automática pelos melhores hiperparâmetros. Na prática, para usar esse método precisamos:

1. Dividir os dados em treino e teste. Lembrando que, para conjuntos maiores $(>1 MM)$ dividimos em treino, teste e validação;
2. Definir uma malha de hiperparâmetros (checar na biblioteca original os hiperparâmetros dos algoritmos a serem utilizados);
3. Definir o tipo de validaçao cruzada (para classificação, usar [`StratifiedKFold`](https://towardsdatascience.com/how-to-train-test-split-kfold-vs-stratifiedkfold-281767b93869));
4. Definir um método de busca (até agora vimos que [`Grid Search`](https://medium.datadriveninvestor.com/an-introduction-to-grid-search-ff57adcc0998) e hoje veremos [`Randomized Search`](https://towardsdatascience.com/random-search-vs-grid-search-for-hyperparameter-optimization-345e1422899d)).

#### Para cada combinação de hiperparâmetros definidos pelas malhas, o algoritmo realiza uma validação cruzada, o que lhe permitirá estimar o erro em produção para aquele conjunto específico. No final, ele nos retorna o conjunto de hiperparâmetros para o qual o erro estimado é mínimo.

#### No final, devemos sempre treinar nossos dados no `X_train` e avaliar a performance no `X_test`, ainda não visto. Essa será a performance estimada em produção.

- OBS 1 : Caso o conjunto de dados seja relativamente grande, não será realizada [validação cruzada](https://towardsdatascience.com/why-and-how-to-cross-validate-a-model-d6424b45261f). Apenas um processo iterativo e automático de busca será realizado. Nesse caso, o algoritmo retorna pra gente a combinação que produz o menor erro no conjunto de validação. No final, treinamos o modelo no `X_train` e estimamos a performance no final utilizando o `X_test`.

- OBS 2: Em um conjunto menor de dados, seria interessante utilizar um método de validação conhecido como [`Nested Cross Validation`](https://weina.me/nested-cross-validation/).

- OBS 3: No caso de dados grandes, não precisamos da validaçao cruzada, então não faz sentido utilizar `GridSearchCV` ou `RandomizedSearchCV`. Podemos, em vez disso, utilizar as classes [`ParameterGrid`](https://www.kdnuggets.com/2018/01/managing-machine-learning-workflows-scikit-learn-pipelines-part-2.html) e [`ParameterSampler`](https://towardsdatascience.com/automated-machine-learning-using-python3-7-improving-efficiency-in-model-development-8c3574febc0b) do [`sklearn`](https://machinelearningmastery.com/scikit-optimize-for-hyperparameter-tuning-in-machine-learning/) para realizar nosso processo de busca.

# ParameterGrid

#### Definimos a `grid` de parâmetros da mesma forma que aprendemos, mas não usaremos `GridSearchCV`. Em vez disso, utilizaremos a clase `ParameterGrid`.

In [90]:
import pandas as pd
from sklearn.model_selection import ParameterGrid
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#### Definimos uma rede de parâmetros.

In [91]:
param_grid = {'n_neighbors': [1, 2]}

In [92]:
param_grid

{'n_neighbors': [1, 2]}

#### A partir daí, criamos o grid com todas as combinações

In [93]:
ParameterGrid(param_grid)

In [94]:
df = pd.read_csv('iris.csv')
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [95]:
type(df)

pandas.core.frame.DataFrame

#### Realizamos a separação do `dataframe` em subconjuntos de treino e teste.

In [96]:
df_train, df_test = train_test_split(df, 
                                   stratify = df['species'], 
                                   test_size = 0.15, 
                                   random_state = 123
                                  )

#### Vamos criar nosso conjunto de validação a partir do conjunt ode treino.

In [97]:
n_rows_validation = int(0.15 * df.shape[0])
percent_validation = round((n_rows_validation / df_train.shape[0]), 2)

print('df_diabetes.shape[0]:', df.shape[0])
print('n_rows_validation:', n_rows_validation)
print('percent_validation:', percent_validation)

df_diabetes.shape[0]: 150
n_rows_validation: 22
percent_validation: 0.17


In [98]:
df_train, df_val = train_test_split(df_train, 
                                    stratify = df_train['species'], 
                                    test_size = percent_validation, 
                                    random_state = 123
                                   )

#### Instanciamos um objeto que recebe o modelo de kNN e separamos nossos subconjuntos de treino `train`, teste `test` e validação `val`. 

In [99]:
knn = KNeighborsClassifier()

In [100]:
X_train, y_train = df_train.drop('species', axis = 1), df_train['species']
X_test, y_test = df_test.drop('species', axis = 1), df_test['species']
X_val, y_val = df_val.drop('species', axis = 1), df_val['species']

print(len(X_train), len(y_train))
print(len(X_test), len(y_test))
print(len(X_val), len(y_val))

105 105
23 23
22 22


In [101]:
validation_error = []
for combinacao in ParameterGrid(param_grid):
    knn.fit(X_train, y_train)
    y_val_calc = knn.predict(X_train)    
    y_val = knn.predict(X_val)
    y_val_calc_sample = pd.DataFrame(y_val_calc).head(n_rows_validation)
    erro = accuracy_score(y_val_calc_sample, y_val)

    #y_pred = knn.predict(X_train)    
    #erro = accuracy_score(y_pred, y_val)
    
    validation_error.append(erro)
    print('combinacao:', combinacao)
    print('validation_error:', validation_error)

combinacao: {'n_neighbors': 1}
validation_error: [0.4090909090909091]
combinacao: {'n_neighbors': 2}
validation_error: [0.4090909090909091, 0.4090909090909091]


In [102]:
type(y_val_calc)

numpy.ndarray

#### Na prática, não faremos validação cruzada. Apenas iremos treinar e testar um modelo para cada combinação do dicionário acima e checar qual retorna para nós o menor erro de validação. Com isso, treinamos nosso modelo no`X_train` e testamos sua performance dele no `X_Test`.

# Randomized Search CV

#### O [`Grid Search`](https://towardsdatascience.com/gridsearch-the-ultimate-machine-learning-tool-6cd5fb93d07) pode ser considerado um método para que escolhe o melhor conjunto de hiperparâmetros por exaustão. No `Grid Search`, o cientista de dados configura uma grade de valores de hiperparâmetros e, para cada combinação, treina um modelo e pontua nos dados de teste. Nessa abordagem, todas as combinações de valores de hiperparâmetros são testadas, o que pode ser muito ineficiente. Por exemplo, pesquisar $20$ valores de parâmetros diferentes para cada um dos $4$ parâmetros exigirá $160,000$ tentativas de validação cruzada. Isso equivale a $1,600,000$ ajustes de modelo e $1,600,000$ previsões se a validação cruzada de $10$ dobras for usada. Embora o [`Scikit Learn`](https://scikit-learn.org/stable/tutorial/statistical_inference/index.html) ofereça a função `GridSearchCV` para simplificar o processo, seria uma execução extremamente cara em termos de capacidade e tempo de computação.

#### Em contraste,o [`Randomized Search`](https://vitalflux.com/randomized-search-explained-python-sklearn-example/) configura uma grade de valores de hiperparâmetros e seleciona combinações aleatórias para treinar o modelo e avaliar o erro. Isso permite que você controle explicitamente o número de combinações de parâmetros que são tentadas. O número de iterações de pesquisa é definido com base no tempo ou recursos. O `Scikit Learn` oferece a função [`RandomizedSearchCV()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html) para esse processo.

#### Embora seja possível que `RandomizedSearchCV` não encontre um resultado tão preciso quanto `GridSearchCV`, ele surpreendentemente escolhe o melhor resultado com mais frequência do que não e em uma fração do tempo que o `GridSearchCV` levaria. Com os mesmos recursos, o `Random Search` pode até superar o `Grid Search`. Isso pode ser visualizado no gráfico abaixo quando parâmetros contínuos são usados.

<img src="gridSearchRandomSearch.png" width=500>

#### O `RandomizedSearch` é baseado em distribuições, ou seja, ele pode selecionar valores entre um intervalo de valores, diferentemente do `GridSearch`.

## ParameterSampler

#### Quando queremos usar uma busca aleatória (`Random Search`), mas sem validação cruzada, podemos usar a classe [`ParameterSampler()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ParameterSampler.html) do `sklearn`. Essa classe vai receber o dicionário com as distribuições dos parâmetros e o número de iterações. 

#### A cada iteração em um laço (`loop`), ele vai gerar um valor aleatório de combinação de hiperparâmetros.

In [103]:
from sklearn.model_selection import ParameterSampler
from scipy.stats import loguniform, uniform, randint

In [104]:
param_grid = dict()
param_grid['solver'] = ['newton-cg', 
                        'lbfgs', 
                        'liblinear'
                       ]
param_grid['penalty'] = ['none', 
                         'l1', 
                         'l2', 
                         'elasticnet'
                        ]
param_grid['C'] = loguniform(1e-5, 
                             100
                            )
param_grid['whatever'] = randint(1, 
                                 1000
                                )

In [105]:
ParameterSampler(param_grid, 
                 n_iter = 10, 
                 random_state = 123
                )

In [106]:
for combinacao in ParameterSampler(param_grid, n_iter = 10, random_state = 123):
    print(combinacao)

{'C': 0.750385268090156, 'penalty': 'l2', 'solver': 'liblinear', 'whatever': 989}
{'C': 0.6857944800896927, 'penalty': 'l1', 'solver': 'liblinear', 'whatever': 124}
{'C': 2.8853223070403433, 'penalty': 'l1', 'solver': 'newton-cg', 'whatever': 114}
{'C': 0.023255373037796706, 'penalty': 'l1', 'solver': 'newton-cg', 'whatever': 943}
{'C': 0.24616089675634506, 'penalty': 'l1', 'solver': 'liblinear', 'whatever': 254}
{'C': 0.7299384488053402, 'penalty': 'none', 'solver': 'newton-cg', 'whatever': 818}
{'C': 0.00018942710113933008, 'penalty': 'l2', 'solver': 'newton-cg', 'whatever': 40}
{'C': 0.9689720939864422, 'penalty': 'elasticnet', 'solver': 'newton-cg', 'whatever': 958}
{'C': 8.831257714012057, 'penalty': 'l1', 'solver': 'newton-cg', 'whatever': 861}
{'C': 1.140522028920257, 'penalty': 'l1', 'solver': 'lbfgs', 'whatever': 631}


## Busca Bayesiana

#### Por último, iremos falar da [`busca bayesiana`](). [Thomas Bayes](https://www.britannica.com/biography/Thomas-Bayes) trouxe diversas contribuições para a estatística, a busca bayesiana vai se basear em probabilidades e em atualização dessa probabilidade, tal como no [teorema de bayes]().

<img src="BayesianSearch.png" width=800>

#### O grande ponto de falha dos métodos que vimos é que eles não levam em consideração erros do passado. Se uma determinada região do espaço trouxe performances abaixo do esperado, é razoável parar de procurar naquela região.

#### As abordagens bayesianas, em contraste com o `Random Search` ou `Grid Search`, rastreiam os resultados de avaliações anteriores que usam para formar um modelo probabilístico de mapeamento de hiperparâmetros. Dessa forma, ao criar esse modelo probabilístico, o algoritmo consegue estimar o ponto no espaço em que a probabilidade de obter um erro menor é maior. 

#### Na literatura, esse modelo é chamado de [`surrogate`]() da função objetivo e é representado como $P(y | x)$.

# Vamos fazer uma aplicação.

In [107]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split, StratifiedKFold
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
%matplotlib inline

#### Separamos nosso `dataset` em treino e teste.

In [108]:
df = load_iris()
X = df.data
y = df.target

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size = 0.20, 
                                                    random_state = 123
                                                   )
len(X_train), len(X_test), len(y_train), len(y_test)

(120, 30, 120, 30)

#### Aplicaremos o método [`StratifiedKFold()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html), um objeto de validação cruzada, que é uma variação de `KFold`, que retorna dobras estratificadas. As dobras são feitas preservando-se o percentual de amostras de cada classe.

In [109]:
# stratified kfold
skf = StratifiedKFold(n_splits = 10, 
                      shuffle = True, 
                      random_state = 123
                     )

In [110]:
from sklearn.model_selection import RandomizedSearchCV

#### Instanciamos um modelo kNN.

In [111]:
param_grid = dict(n_neighbors = range(1, 31))
knn = KNeighborsClassifier()

In [112]:
param_grid

{'n_neighbors': range(1, 31)}

#### E realizamos uma busca aleatórea da melhor combinação de hiperparâmetros com o método [`RandomizedSearchCV()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html).

In [113]:
# random search
#%time
random_search = RandomizedSearchCV(knn, 
                                   param_grid, 
                                   n_iter = 10,
                                   cv = skf, 
                                   scoring = 'neg_log_loss', 
                                   verbose = 1, 
                                   random_state = 123
                                  )

In [114]:
random_search.fit(X_train, y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.6s finished


RandomizedSearchCV(cv=StratifiedKFold(n_splits=10, random_state=123, shuffle=True),
                   estimator=KNeighborsClassifier(),
                   param_distributions={'n_neighbors': range(1, 31)},
                   random_state=123, scoring='neg_log_loss', verbose=1)

In [115]:
random_search.best_params_

{'n_neighbors': 12}

In [116]:
random_search.best_score_

-0.07803933303339768

In [117]:
# grid search
#%time
grid_search = GridSearchCV(knn, 
                           param_grid, 
                           cv = skf, 
                           scoring = 'neg_log_loss', 
                           verbose = 1
                          )

In [118]:
grid_search.fit(X_train, y_train)

Fitting 10 folds for each of 30 candidates, totalling 300 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    1.0s finished


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=123, shuffle=True),
             estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': range(1, 31)}, scoring='neg_log_loss',
             verbose=1)

In [119]:
grid_search.best_params_

{'n_neighbors': 12}

In [120]:
grid_search.best_score_

-0.07803933303339768

#### Note que o `Grid search` demorou mais que o dobro de tempo para rodar. No final, o `Random search` conseguiu encontrar o melhor número de vizinhos mais próximos, com menos modelos.